In [1]:
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.helpers import DataConnection, S3Location
import ibm_boto3
from ibm_watsonx_ai import APIClient
from ibm_watsonx_ai.foundation_models.extractions import TextExtractions
from ibm_watsonx_ai.metanames import TextExtractionsMetaNames
import time

In [71]:
from ibm_watsonx_ai import APIClient

credentials = {
    "apikey": "wx388gAQyq849mgWAd6Ph7eMXNXK_vomMbJMI_3zlXG9",
    "url": "https://eu-de.ml.cloud.ibm.com"
}

project_id = "61249d5d-fa8e-4501-b6c4-8aa4156e48b0"  # Replace with your actual Watsonx project ID

wx_client = APIClient(credentials=credentials, project_id=project_id)


In [28]:
connection_asset_id = "0ef0fd193d1b4036a9662442f83e7638"
bucketname = "textextraction-donotdelete-pr-nh2q6azrylxq1a"

In [41]:
CloudObjectStorage_client = ibm_boto3.client(
    service_name='s3',
    aws_access_key_id='0ef0fd193d1b4036a9662442f83e7638',
    aws_secret_access_key='96f5a65862b6845af03166691548d2adef4a80a8f7ba6081',
    endpoint_url='https://s3.eu-de.cloud-object-storage.appdomain.cloud',  # You can change the region as needed
    config=Config(signature_version='s3v4')
)
bucketname = "textextraction-donotdelete-pr-nh2q6azrylxq1a"

In [42]:
response = CloudObjectStorage_client.list_buckets()
for bucket in response['Buckets']:
    print(bucket['Name'])



asmitasarkarssandbox-donotdelete-pr-0hktbca5oiapso
new-cloud-object-storage-cos-standard-7px
textextraction-donotdelete-pr-nh2q6azrylxq1a
wml2project-donotdelete-pr-nnyc9qn9fjwoik
wmlproject-donotdelete-pr-qnbctnp93ye6kx


In [63]:
cos_credentials = {
                "endpoint_url": "https://s3.eu-de.cloud-object-storage.appdomain.cloud",
                "apikey": "McNCj4qlk-AkkHZ9VwCZGTjz4sf-FI9eRZQnvvghA5B6",
                "access_key_id": "0ef0fd193d1b4036a9662442f83e7638",
                "secret_access_key": "96f5a65862b6845af03166691548d2adef4a80a8f7ba6081"
            }

In [64]:
conn_meta_props = {
    client.connections.ConfigurationMetaNames.NAME: "Connection to Database - bluemixcloudobjectstorage",
    client.connections.ConfigurationMetaNames.DATASOURCE_TYPE: client.connections.get_datasource_type_id_by_name("bluemixcloudobjectstorage"),
    client.connections.ConfigurationMetaNames.DESCRIPTION: "Connection to external Database",
    client.connections.ConfigurationMetaNames.PROPERTIES: {
        'bucket': bucketname,
        'access_key': cos_credentials['access_key_id'],
        'secret_key': cos_credentials['secret_access_key'],
        'iam_url': 'https://iam.cloud.ibm.com/identity/token',
        'url': cos_credentials['endpoint_url']
    }
}


In [65]:
conn_details = client.connections.create(meta_props=conn_meta_props)
connection_asset_id = client.connections.get_id(conn_details)
print(connection_asset_id)

Creating connections...
SUCCESS
3f3139a1-c4fb-4ca5-81b1-26c651eebd5d


In [76]:
def text_extraction(file_names, extraction, steps, results_path):
    for source_file_name in file_names:
        if source_file_name == '.DS_Store':
            continue

        results_file_name = "text_extracted_" + source_file_name.replace("pdf", "json")

        connection_asset_id = "3f3139a1-c4fb-4ca5-81b1-26c651eebd5d"  # <-- REPLACE THIS
        bucketname = "textextraction-donotdelete-pr-nh2q6azrylxq1a"  # <-- REPLACE THIS

        document_reference = DataConnection(
            connection_asset_id=connection_asset_id,
            location=S3Location(bucket=bucketname, path=source_file_name)
        )

        results_reference = DataConnection(
            connection_asset_id=connection_asset_id,
            location=S3Location(bucket=bucketname, path=results_file_name)
        )

        document_reference.set_client(wx_client)
        results_reference.set_client(wx_client)

        try:
            # Create extraction job
            details = extraction.run_job(
                document_reference=document_reference,
                results_reference=results_reference,
                steps=steps,
                results_format="markdown"
            )

            extraction_job_id = extraction.get_id(extraction_details=details)
            print("\n" + source_file_name + " - " + extraction_job_id)

            while True:
                status_json = extraction.get_job_details(extraction_id=extraction_job_id)
                status = status_json['entity']['results']['status']
                print(status)
                if status == "failed":
                    print(status_json)
                    break
                if status != "completed":
                    time.sleep(5)
                else:
                    break

            if status == "completed":
                final_results_reference = extraction.get_results_reference(extraction_id=extraction_job_id)
                filename = source_file_name.replace("pdf", "md")
                final_results_reference.download(results_path + "/" + filename)
                print("saved as " + filename)

        except Exception as e:
            print("error: ", e)

    return "done"


In [80]:
# calling text_extraction function

extraction = TextExtractions(api_client=wx_client, project_id=project_id)

steps = {TextExtractionsMetaNames.OCR: {'languages_list': ['en']},
        TextExtractionsMetaNames.TABLE_PROCESSING: {'enabled': True}}

In [ ]:
file_names = ["Paper-IUGRC-1021.pdf"]
results_path = "./output"

text_extraction(file_names, extraction, steps, results_path)
